## Install and import necessary Modules

In [1]:
!conda install azure --y

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /home/nbcommon/anaconda3_420:

The following NEW packages will be INSTALLED:

    azure:    1.0.2-py35_0
    readline: 6.2-2       

readline-6.2-2 100% |################################| Time: 0:00:00   5.83 MB/s
azure-1.0.2-py 100% |################################| Time: 0:00:00   9.48 MB/s


In [2]:
!pip install azure.storage

    100% |################################| 194kB 7.6MB/s eta 0:00:01


In [3]:
import os
import platform
import json
import pandas as pd
import urllib.parse
import http.client
import base64
from azure.storage.table import TableBatch, TableService, Entity
import time

# Connect to Sign With Robert Giphy Channel
![](https://media.giphy.com/media/3o7TKNKOfKlIhbD3gY/giphy.gif)

In [4]:
subscription_key = 'b8e8f4ccd0b24398a55f457d8f91bc4d'

uri_base = 'eastus2.api.cognitive.microsoft.com/vision/v1.0'

headers = {
    # Request headers.
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': subscription_key,
}

params = urllib.parse.urlencode({
    # Request parameters. The language setting "unk" means automatically detect the language.
    'language': 'en'
})


In [5]:
def getDataStill(url, urlstill):

    # The URL of a JPEG image containing text.
    
    body = "{'url': '%s'}" % urlstill
#     print urlstill

    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("POST", "/vision/v1.0/ocr?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read().decode()
        val = getText(data,url,urlstill)
        
        return val
        conn.close()

    except Exception as e:
        try:
            getData(url,urlstill)
        except Exception as e2:
#             print(error)
            return("")

def getData(url,urlstill):
    body = "{'url': '%s'}" % url

    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("POST", "/vision/v1.0/ocr?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read().decode()
        val = getText(data,url,urlstill)
        return val
        conn.close()

    except Exception as e:
#         print(e)
        return("")

limit =0
    
def getText(data,url,urlstill):
    text = ""
    global limit
    if data is not "":
        df1 = pd.read_json(data)
        
        if df1['regions'] is not None:
            for i in df1['regions']:
                val2 = i['lines']
                for i in val2:
                    for j in i['words']:
                     text += j['text'] + " "
                if text is "":
                    if limit is 0:
                        limit +=1
                        getDataStill(url,urlstill)
                    else:
                        text = ""
                        limit = 0
    else:
        if limit ==0:
            limit += 1
            getDataStill(url,urlstill)
        else:
            text = "" 
            limit = 0   
    limit = 0        
    # time.sleep(0.3)
    text = spellCheck(text)
    print(text)
    return(text)

num= 5

In [6]:
spellCheckKey = 'd3a9f828595d448c97ce96423104fbe6'

uri_baseSpell = 'api.cognitive.microsoft.com'

headers2 = {
    # Request headers.
    'Content-Type': 'application/x-www-form-urlencoded',
    'Ocp-Apim-Subscription-Key': spellCheckKey,
}

params2 = urllib.parse.urlencode({
    # Request parameters. The language setting "unk" means automatically detect the language.
    'language': 'en',

})

def spellCheck(text):
    if text is "":
        return ""
    body2 = "text=" + text.replace(" ", "+")
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection('api.cognitive.microsoft.com')
        conn.request("POST", "/bing/v7.0/spellcheck/?%s" % params2, body2, headers2)
        response = conn.getresponse()
        data = response.read().decode()
        val = json.loads(data)
        if len(val['flaggedTokens'])>0:
            return correctText(val,text)
        else:
            return text
        conn.close()

    except Exception as e:
        return ""
    

In [7]:
def correctText(val, text):
    for i in val['flaggedTokens']:
        token = i["token"]
        vals= i["suggestions"][0]["suggestion"]
        text = text.replace(token, vals)
    return text

In [23]:
import urllib.request
import codecs

giphy_Api_key = 'Enter Giphy api key'
giphy = "http://api.giphy.com/v1/gifs/search?q=sign+with+robert&api_key=" + giphy_Api_key +  "&limit=500&offset="

count = 0
url = ""
def getUrl(offset):
    contents = urllib.request.urlopen(giphy+str(offset))
    reader = codecs.getreader("utf-8")
    jsonContent = json.load(reader(contents))
    global count
    count+=1
    for i in jsonContent['data']:
        
        url = i['images']['fixed_height']['url']
        urlstill = i['images']['fixed_height_still']['url']
        if url is '' and urlstill is '':
            count -=1
            return
        else:
            listdict.append(url)
            stilldict.append(urlstill)
#         print(count)
            count+=1

In [24]:
df = pd.DataFrame()
listdict = []
textDict = []
stilldict = []

getUrl(0)
getUrl(500)
getUrl(1000)
getUrl(1500)

In [10]:
df['url'] = listdict
df['urlstill'] = stilldict
df = df[df.url != '']
df = df[df.urlstill !='']

In [25]:
df2 = pd.read_csv("outputTableSpellCheck.csv")

In [26]:
df2 = df2[df2['phrase']!=""]
df2 = df2[df2['phrase']!="Nan"]
df2 = df2.reset_index(drop=True)
df2['phrase'] = df2['phrase'].apply(lambda x: str(x).capitalize())
df2 = df2[df2['phrase']!="Nan"]
df2 = df2.drop_duplicates()

In [16]:
df2

,PartitionKey,RowKey,Timestamp,url,url@type,phrase,phrase@type,urlstill,urlstill@type
0,tasksSign,1,2017-09-28T18:30:01.640Z,https://media1.giphy.com/media/l0MYxJxWgnNaXe2...,Edm.String,Alcohol,Edm.String,https://media1.giphy.com/media/l0MYxJxWgnNaXe2...,Edm.String
1,tasksSign,2,2017-09-28T18:30:01.640Z,https://media0.giphy.com/media/26DOMxDhzBIet0Y...,Edm.String,Dead,Edm.String,https://media0.giphy.com/media/26DOMxDhzBIet0Y...,Edm.String
3,tasksSign,4,2017-09-28T18:30:01.640Z,https://media2.giphy.com/media/3o7TKTtBYWU4Iey...,Edm.String,Popcorn,Edm.String,https://media2.giphy.com/media/3o7TKTtBYWU4Iey...,Edm.String
4,tasksSign,5,2017-09-28T18:30:01.640Z,https://media3.giphy.com/media/3o6ZtmnidXHIoQJ...,Edm.String,Cinco de mayo,Edm.String,https://media3.giphy.com/media/3o6ZtmnidXHIoQJ...,Edm.String
5,tasksSign,6,2017-09-28T18:30:01.640Z,https://media0.giphy.com/media/3o7TKGggEJ1AgD3...,Edm.String,Fat overweight,Edm.String,https://media0.giphy.com/media/3o7TKGggEJ1AgD3...,Edm.String
6,tasksSign,7,2017-09-28T18:30:01.640Z,https://media0.giphy.com/media/l0MYQOXZzNvdwEk...,Edm.String,Wine,Edm.String,https://media0.giphy.com/media/l0MYQOXZzNvdwEk...,Edm.String
7,tasksSign,8,2017-09-28T18:30:01.640Z,https://media2.giphy.com/media/l4q88nfCo5ipr79...,Edm.String,Sick,Edm.String,https://media2.giphy.com/media/l4q88nfCo5ipr79...,Edm.String
8,tasksSign,9,2017-09-28T18:30:01.640Z,https://media0.giphy.com/media/l0MYzjlxBWJn24W...,Edm.String,"Here's an example to shoo someone""ay. bye bye.",Edm.String,https://media0.giphy.com/media/l0MYzjlxBWJn24W...,Edm.String
9,tasksSign,10,2017-09-28T18:30:01.640Z,https://media3.giphy.com/media/l0MYALSY7psvvVs...,Edm.String,"""are you alright?'",Edm.String,https://media3.giphy.com/media/l0MYALSY7psvvVs...,Edm.String
10,tasksSign,11,2017-09-28T18:30:01.640Z,https://media0.giphy.com/media/26DOMIFuQMlhmuS...,Edm.String,Alive,Edm.String,https://media0.giphy.com/media/26DOMIFuQMlhmuS...,Edm.String


## Find Closest Match

In [17]:
!pip install fuzzywuzzy[speedup]

    100% |################################| 51kB 1.6MB/s ta 0:00:011
  Running setup.py bdist_wheel for python-levenshtein ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/c0/83/e9/b2cc2876e175d04091caf4e9f5de564ff2503b1f1885e7c3ba
Successfully built python-levenshtein


In [18]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [19]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

process.extract("what are you doing", df2['phrase'])

[('What are you doing?', 100, 209),
 ('Hat', 90, 573),
 ('Hat', 90, 1898),
 ('What are 90u doing?', 89, 858),
 ('How are you doing?', 86, 66)]

In [20]:
URL=str(df2[df2['phrase'].isin(process.extractOne("Wheres bathroom", df2['phrase']))]['url'].values[0])
print(URL)
display_gif(URL)

https://media2.giphy.com/media/l0MYKTHdq5SE9KV44/200.gif


In [21]:
import difflib

diffUrl = str(df2[df2['phrase'].isin(difflib.get_close_matches("where's bathroom", df2['phrase']))]['url'].values[0])
display_gif(diffUrl)

## Speech to Text